# LLM-Based Eng-to-Prsn Translation System Using In-Context Learning

1. Model Choice (Light LLaMA-Based)

Recommended lightweight LLaMA-family models on Hugging Face:

✅ TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0
✅ NousResearch/Llama-2-7b-chat-hf (if you have GPU ≥16GB)

For a light notebook, we’ll use:

## Install dependencies

In [2]:
!pip install transformers accelerate sentencepiece sacrebleu datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.7 MB/s eta 0:00:00


In [3]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `LLMCourse` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

## Import libraries

In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import sacrebleu

## Load the Pretrained LLaMA Model

In [34]:

MODEL_NAME = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

## Dataset (English → Persian)


*   opus100 (en–fa)

*   Tatoeba

*   Custom CSV

In [35]:
dataset = load_dataset("opus100", "en-fa", split="test[:100]")

In [7]:
len(dataset)

100

In [36]:
dataset[0]["translation"]

{'en': "I'm obsolete!", 'fa': 'من دیگه به دردی نمیخورم'}

## In-Context Learning Prompt Design

Prompt Template

few-shot examples are provided inside the prompt.

In [95]:
def build_prompt(examples, source_sentence):
    prompt = "Translate the following sentences from English to Persian.\n\n"

    for ex in examples:
        prompt += f"English: {ex['en']}\n"
        prompt += f"Persian: {ex['fa']}\n\n"

    prompt += f"English: {source_sentence}\nPersian:"
    return prompt


In [97]:
n = 3
icl_examples = dataset[0:n]["translation"]

In [98]:
icl_examples

[{'en': "I'm obsolete!", 'fa': 'من دیگه به دردی نمیخورم'},
 {'en': "You've really gotta help me now.",
  'fa': 'تو واقعا ديگه بايد کمکم کني.'},
 {'en': 'The entire time you were in rehab,',
  'fa': '\u200fکل مدت\u200cزمانی که توی بازپروری بودی'}]

## Translation Function

In [99]:
def translate(en_text: str):
    tokenizer.src_lang = "eng_Latn"

    inputs = tokenizer(
        en_text,
        return_tensors="pt",
        truncation=False,
        max_length=512
    ).to(device)

    forced_bos_token_id = tokenizer.convert_tokens_to_ids("pes_Arab")

    generated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos_token_id,
        max_length=512,
        num_beams=5
    )

    return tokenizer.batch_decode(
        generated_tokens,
        skip_special_tokens=True
    )[0]


In [100]:
from tqdm import tqdm

In [101]:
from tqdm import tqdm

references = []
hypotheses = []

for item in tqdm(dataset[n:]["translation"], desc="NLLB Translating"):
    en = item["en"]
    fa = item["fa"]

    pred = translate(en)

    references.append([fa])
    hypotheses.append(pred)


NLLB Translating: 100%|██████████| 97/97 [00:24<00:00,  3.92it/s]


In [102]:
print(f"{references[1]} \n {hypotheses[1]}")

['لباس قبلي هامو جا گذاشتم'] 
 لباس های دیگه ام رو داخل گذاشتم.


## BLEU Score Evaluation

In [103]:
bleu = sacrebleu.corpus_bleu(hypotheses, references)
print(f"BLEU score: {bleu.score:.2f}")

BLEU score: 19.30
